In [185]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from tkinter import filedialog
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tkinter import messagebox 
import sqlite3



In [186]:
tablo1_list=[]
tablo2_list=[]
tablo3_list=[]
tablo2_agirliklar=[]
tablo2_basliklar=[]
tablo2_sırano=[]
ogrenci_notlari_tablosu=[]
ogrenci_basarilari=[]

In [187]:
# Veritabanı bağlantısı oluştur
conn = sqlite3.connect("Proje3Database.db")
cursor = conn.cursor()

# Dersler tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS Dersler (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Adi TEXT NOT NULL,
    Kodu TEXT NOT NULL
)
""")


# Öğrenciler tablosu
cursor.execute("""
CREATE TABLE IF NOT EXISTS Ogrenciler (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    No TEXT UNIQUE
)
""")
# Dönem içi çıktı tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS DersCikti (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    SiraNo INTEGER NOT NULL,
    Icerik TEXT NOT NULL,
    Ders_id INTEGER NOT NULL,
    FOREIGN KEY (Ders_id) REFERENCES Dersler (id)
)
""")

# Program çıktı tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS ProgramCikti (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    SiraNo INTEGER NOT NULL,
    Icerik TEXT NOT NULL,
    Ders_id INTEGER NOT NULL,
    FOREIGN KEY (Ders_id) REFERENCES Dersler (id)
)
""")

# Notlar tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS Notlar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    OgrenciId INTEGER NOT NULL,
    DersId INTEGER NOT NULL,
    Puan INTEGER NOT NULL,
    Tur TEXT NOT NULL,
    FOREIGN KEY (DersId) REFERENCES Dersler (id),
    FOREIGN KEY (OgrenciId) REFERENCES Ogrenciler (id)
)
""")

#  Puan turu tablosu oluştur
cursor.execute("""
CREATE TABLE IF NOT EXISTS PuanTuru (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tur TEXT NOT NULL,
    DersId INTEGER NOT NULL,
    Yuzdesi INTEGER Not NULL,
    FOREIGN KEY (DersId) REFERENCES Dersler (id)
    
)
""")

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
conn.close()

print("Veritabanı ve tablolar başarıyla oluşturuldu.")


Veritabanı ve tablolar başarıyla oluşturuldu.


In [188]:
def ekle_butonu_action():
    # Ders adı ve ders kodunu al
    ders_adi = ders_adi_entry.get()
    ders_kodu = ders_kodu_entry.get()
    
    # Eğer alanlar boşsa uyarı ver
    if not ders_adi or not ders_kodu:
        tk.messagebox.showwarning("Eksik Bilgi", "Lütfen tüm alanları doldurunuz.")
        return
    
    try:
        # Veritabanına bağlan
        conn = sqlite3.connect("Proje3Database.db")
        cursor = conn.cursor()
        
        # Girilen bilgileri veritabanına ekle
        cursor.execute("INSERT INTO Dersler (Adi, Kodu) VALUES (?, ?)", (ders_adi, ders_kodu))
        
        # Değişiklikleri kaydet ve bağlantıyı kapat
        conn.commit()
        conn.close()
        
        # Başarı mesajı
        tk.messagebox.showinfo("Başarılı", "Ders başarıyla eklendi.")
        
        # Alanları temizle
        ders_adi_entry.delete(0, tk.END)
        ders_kodu_entry.delete(0, tk.END)
        
    except Exception as e:
        tk.messagebox.showerror("Hata", f"Bir hata oluştu: {e}")
    
    ders_sec_combobox['values'] = dersleri_listeye_al()


In [189]:
def ogrenci_ekle_action():

    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()

    try:
       
        df = pd.read_excel(dosya_sec_entry.get())

        # 'Ogrenci_No' sütununun varlığını kontrol et
        if "Ogrenci_No" not in df.columns:
            print("Hata: Excel dosyasında 'Ogrenci_No' adlı bir sütun bulunamadı.")
            return

        # "Ogrenci_No" sütununu al, boş değerleri çıkar, string'e çevir ve benzersiz değerleri belirle
        ogrenciler = df["Ogrenci_No"].dropna().astype(str).unique()

        # Veritabanına ekle
        for no in ogrenciler:
            try:
                cursor.execute("INSERT INTO Ogrenciler (No) VALUES (?)", (no,))
            except sqlite3.IntegrityError:
                print(f"Öğrenci numarası zaten mevcut: {no}")

        conn.commit()
        print("Excel dosyasındaki öğrenci numaraları veritabanına başarıyla eklendi.")
        tk.messagebox.showinfo("Başarılı", "Ogrenciler başarıyla eklendi.")
    except Exception as e:
        print(f"Hata: {e}")
    finally:
        conn.close()


In [190]:
def dersleri_listeye_al():
    try:
        # Veritabanına bağlan
        conn = sqlite3.connect("Proje2Database.db")  # Veritabanı dosyasının adı
        cursor = conn.cursor()
        
        # Dersleri veritabanından çek
        cursor.execute("SELECT Id, Adi, Kodu FROM Dersler")
        dersler = cursor.fetchall()  # [(ders_adi1, ders_kodu1), (ders_adi2, ders_kodu2), ...]

        # Listeye dönüştürme
        ders_listesi = [f"{ders[0]} ({ders[1]}) ({ders[2]})" for ders in dersler]  # Örneğin: "Matematik (MAT101)"
        
        return ders_listesi

    except sqlite3.Error as e:
        print("Veritabanı hatası:", e)
        return []

    finally:
        # Bağlantıyı kapat
        if conn:
            conn.close()

In [191]:
def ders_cikti_ekle_buton_action():
    """
    Seçilen derse ait ders çıktılarını Excel dosyasından okuyup veritabanına ekler.
    """
    conn = None
    try:
        # --- 1. Ders ve Dosya Seçimi ---
        secilen_ders = ders_sec_combobox.get()
        secilen_ders = secilen_ders.split(" ") [0]
        print(f"Seçilen Ders: {secilen_ders}")

        if not secilen_ders:
            messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")
            return

        dosya_adi = dosya_sec_entry.get()
        print(f"Seçilen Dosya: {dosya_adi}")

        if not dosya_adi:
            messagebox.showwarning("Uyarı", "Herhangi bir dosya seçilmedi.")
            return

        # --- 2. Ders ID'sini Bul ---
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        print("Veritabanına bağlanıldı.")

        cursor.execute("SELECT id FROM Dersler WHERE ID = ?", (secilen_ders,))
        ders_id_tuple = cursor.fetchone()
        print(f"Ders ID Tuple: {ders_id_tuple}")

        if not ders_id_tuple:
            messagebox.showerror("Hata", f"'{secilen_ders}' dersi veritabanında bulunamadı.")
            return

        ders_id = ders_id_tuple[0]
        print(f"Ders ID: {ders_id}")

        # --- 3. Excel'den Verileri Oku ---
        df = pd.read_excel(dosya_adi)
        print("Excel dosyası okundu.")

        # Sütun başlıklarını atla ve sadece Sıra No ve İçerik sütunlarını al
        ciktilar = df.iloc[0:].values.tolist()
        print(f"Çıktılar Listesi: {ciktilar}")

        # --- 4. Veritabanına Ekle ---
        for sira_no, icerik in ciktilar:
            cursor.execute(
                "INSERT INTO DersCikti (Ders_id, SiraNo, Icerik) VALUES (?, ?, ?)",
                (ders_id, sira_no, icerik),
            )
            print(f"Eklendi: Ders ID: {ders_id}, Sıra No: {sira_no}, İçerik: {icerik}")

        conn.commit()
        print("Değişiklikler kaydedildi (commit).")
        messagebox.showinfo("Başarılı", "Ders çıktıları başarıyla veritabanına eklendi.")

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")
    except pd.errors.ParserError:
        messagebox.showerror("Hata", "Dosya okuma hatası. Geçerli bir Excel dosyası seçtiğinizden emin olun.")
    except sqlite3.Error as e:
        messagebox.showerror("Hata", f"Veritabanı hatası: {e}")
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")
    finally:
        if conn:
            conn.close()
            print("Veritabanı bağlantısı kapatıldı.")

In [192]:
def program_cikti_ekle_buton_action():
    """
    Seçilen derse ait program çıktıları Excel dosyasından okuyup veritabanına ekler.
    """
    conn = None
    try:
        # --- 1. Ders ve Dosya Seçimi ---
        secilen_ders = ders_sec_combobox2.get().split(" ")[0]  # Ders ID'sini al
        print(f"Seçilen Ders: {secilen_ders}")

        if not secilen_ders:
            messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")
            return

        dosya_adi = dosya_sec_entry2.get()
        print(f"Seçilen Dosya: {dosya_adi}")

        if not dosya_adi:
            messagebox.showwarning("Uyarı", "Herhangi bir dosya seçilmedi.")
            return

        # --- 2. Veritabanı Bağlantısı ---
        conn = sqlite3.connect("Proje2Database.db")
        cursor = conn.cursor()
        print("Veritabanına bağlanıldı.")

        # Ders ID kontrolü
        cursor.execute("SELECT id FROM Dersler WHERE ID = ?", (secilen_ders,))
        ders_id_tuple = cursor.fetchone()
        print(f"Ders ID Tuple: {ders_id_tuple}")

        if not ders_id_tuple:
            messagebox.showerror("Hata", f"'{secilen_ders}' dersi veritabanında bulunamadı.")
            return

        ders_id = ders_id_tuple[0]
        print(f"Ders ID: {ders_id}")

        # --- 3. Excel Dosyasını Oku ---
        try:
            df = pd.read_excel(dosya_adi)
            print("Excel dosyası başarıyla okundu.")
        except Exception as e:
            messagebox.showerror("Hata", f"Excel dosyası okunamadı: {e}")
            return

        # Sıra ve İçerik sütunlarının alınıp listeye dönüştürülmesi
        if "Sıra" not in df.columns or "Program Öğrenme Çıktıları" not in df.columns:
            messagebox.showerror("Hata", "Excel dosyası beklenen formatta değil. 'Sıra' ve 'Öğrenme Çıktıları' sütunlarını kontrol edin.")
            return

        ciktilar = df[["Sıra", "Program Öğrenme Çıktıları"]].values.tolist()
        print(f"Çıktılar Listesi: {ciktilar}")

        # --- 4. Veritabanına Verileri Ekle ---
        for sira_no, icerik in ciktilar:
            cursor.execute(
                "INSERT INTO ProgramCikti (Ders_id, SiraNo, Icerik) VALUES (?, ?, ?)",
                (ders_id, sira_no, icerik),
            )
            print(f"Eklendi: Ders ID: {ders_id}, Sıra No: {sira_no}, İçerik: {icerik}")

        conn.commit()
        print("Değişiklikler kaydedildi.")
        messagebox.showinfo("Başarılı", "Program çıktıları başarıyla veritabanına eklendi.")

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")
    except sqlite3.Error as e:
        messagebox.showerror("Hata", f"Veritabanı hatası: {e}")
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")
    finally:
        if conn:
            conn.close()
            print("Veritabanı bağlantısı kapatıldı.")


In [193]:


# Ana pencereyi oluştur
pencere = tk.Tk()
pencere.title("Ders Planı")
pencere.geometry("900x700")

# Notebook (Sekmeler) oluştur
sekme_defteri = ttk.Notebook(pencere)
sekme_defteri.pack(fill=tk.BOTH, expand=True)

# Sekme çerçeveleri
ders_ekleme_sekmesi = ttk.Frame(sekme_defteri)
program_ders_sekmesi = ttk.Frame(sekme_defteri)
tablo_ekleme_sekmesi = ttk.Frame(sekme_defteri)
degerlendirme_sekmesi = ttk.Frame(sekme_defteri)
not_yukleme_sekmesi = ttk.Frame(sekme_defteri)


In [194]:

# Sekmeleri deftere ekle
sekme_defteri.add(ders_ekleme_sekmesi, text="Ders Ekleme ve Öğrenci Listesi")
ders_yukleme_frame = ttk.LabelFrame(ders_ekleme_sekmesi, text="Ders Yükleme")
ders_yukleme_frame.pack(fill="x", padx=20, pady=10)

tk.Label(ders_yukleme_frame, text="Ders Adı:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_adi_entry = ttk.Entry(ders_yukleme_frame)
ders_adi_entry.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(ders_yukleme_frame, text="Ders Kodu:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
ders_kodu_entry = ttk.Entry(ders_yukleme_frame)
ders_kodu_entry.grid(row=0, column=3, padx=10, pady=5, sticky="ew")
ekle_butonu = ttk.Button(ders_yukleme_frame, text="Dersi Yükle",command=ekle_butonu_action)
ekle_butonu.grid(row=0, column=4, padx=10, pady=5)

ogrenci_listesi = ttk.LabelFrame(ders_ekleme_sekmesi, text="Öğrencileri Yükleme")
ogrenci_listesi.pack(fill="x", padx=20, pady=10)

tk.Label(ogrenci_listesi, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry = ttk.Entry(ogrenci_listesi)
dosya_sec_entry.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry.delete(0, tk.END)
    dosya_sec_entry.insert(0, dosya)

dosya_sec_butonu = ttk.Button(ogrenci_listesi, text="...", command=dosya_sec)
dosya_sec_butonu.grid(row=0, column=4, padx=10, pady=5)
ekle_butonu = ttk.Button(ogrenci_listesi, text="Ogrencileri Yükle",command=ogrenci_ekle_action)
ekle_butonu.grid(row=0, column=5, padx=10, pady=5)



dersler_frame = ttk.LabelFrame(ders_ekleme_sekmesi, text="Dersler")
dersler_frame.pack(fill="x", padx=10, pady=10)


try:
    # Veritabanına bağlan
    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()

    # Dersleri sorgula
    cursor.execute("SELECT Adi, Kodu FROM Dersler")
    dersler = cursor.fetchall()  # [(Matematik, MATH101), (Fizik, PHYS201)]
except Exception as e:
    print(f"Hata: {e}")
    dersler = []
finally:
    if conn:
        conn.close()

# Listbox oluştur
ders_listbox = tk.Listbox(dersler_frame, height=6, selectmode="single")
ders_listbox.pack(fill="both", padx=10, pady=10)

# Dersleri Listbox'a ekle
for ders_adi, ders_kodu in dersler:
    ders_listbox.insert(tk.END, f"{ders_adi} ({ders_kodu})")

In [195]:

sekme_defteri.add(program_ders_sekmesi, text="Program ve Ders Çıktıları")
# === Ders Çıktıları Yükleme ===
ders_ciktilari_frame = ttk.LabelFrame(program_ders_sekmesi, text="Ders Çıktıları Yükleme")
ders_ciktilari_frame.pack(fill="x", padx=20, pady=10)

tk.Label(ders_ciktilari_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox = ttk.Combobox(ders_ciktilari_frame, values=dersleri_listeye_al())
ders_sec_combobox.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(ders_ciktilari_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry = ttk.Entry(ders_ciktilari_frame)
dosya_sec_entry.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry.delete(0, tk.END)
    dosya_sec_entry.insert(0, dosya)

dosya_sec_butonu = ttk.Button(ders_ciktilari_frame, text="...", command=dosya_sec)
dosya_sec_butonu.grid(row=0, column=4, padx=10, pady=5)

ders_cikti_ekle_butonu = ttk.Button(ders_ciktilari_frame, text="Ders Çıktılarını Yükle", command=ders_cikti_ekle_buton_action)
ders_cikti_ekle_butonu.grid(row=0, column=5, padx=10, pady=5)

# === Program Çıktıları Yükleme ===
program_ciktilari_yukleme_frame = ttk.LabelFrame(program_ders_sekmesi, text="Program Çıktıları Yükleme")
program_ciktilari_yukleme_frame.pack(fill="x", padx=20, pady=10)

tk.Label(program_ciktilari_yukleme_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox2 = ttk.Combobox(program_ciktilari_yukleme_frame, values=dersleri_listeye_al())
ders_sec_combobox2.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

tk.Label(program_ciktilari_yukleme_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry2 = ttk.Entry(program_ciktilari_yukleme_frame)
dosya_sec_entry2.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec2():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry2.delete(0, tk.END)
    dosya_sec_entry2.insert(0, dosya)

dosya_sec_butonu2 = ttk.Button(program_ciktilari_yukleme_frame, text="...", command=dosya_sec2)
dosya_sec_butonu2.grid(row=0, column=4, padx=10, pady=5)

program_cikti_ekle_butonu = ttk.Button(program_ciktilari_yukleme_frame, text="Program Çıktılarını Yükle", command=program_cikti_ekle_buton_action)
program_cikti_ekle_butonu.grid(row=0, column=5, padx=10, pady=5)

from tkinter import ttk, filedialog, messagebox
import tkinter as tk

def create_dynamic_entry_fields():
    def add_new_row():
        row_idx = len(dynamic_entry_fields)
        sayi_entry = ttk.Entry(dynamic_fields_frame, width=10)
        sayi_entry.grid(row=row_idx + 1, column=0, padx=10, pady=5)
        ogrenme_ciktisi_entry = ttk.Entry(dynamic_fields_frame, width=60)
        ogrenme_ciktisi_entry.grid(row=row_idx + 1, column=1, padx=10, pady=5)
        delete_button = ttk.Button(dynamic_fields_frame, text="Sil", command=lambda: delete_row(row_idx))
        delete_button.grid(row=row_idx + 1, column=2, padx=10, pady=5)
        dynamic_entry_fields.append((sayi_entry, ogrenme_ciktisi_entry, delete_button))

    def delete_row(row_idx):
        entry_set = dynamic_entry_fields.pop(row_idx)
        for widget in entry_set:
            widget.destroy()
        # Satırları yeniden düzenle
        for i, (sayi_entry, ogrenme_ciktisi_entry, delete_button) in enumerate(dynamic_entry_fields):
            sayi_entry.grid(row=i + 1, column=0, padx=10, pady=5)
            ogrenme_ciktisi_entry.grid(row=i + 1, column=1, padx=10, pady=5)
            delete_button.grid(row=i + 1, column=2, padx=10, pady=5)

    def collect_and_process_data():
        collected_data = []
        for sayi_entry, ogrenme_ciktisi_entry, _ in dynamic_entry_fields:
            sayi = sayi_entry.get().strip()
            ogrenme_ciktisi = ogrenme_ciktisi_entry.get().strip()
            if sayi and ogrenme_ciktisi:
                collected_data.append((sayi, ogrenme_ciktisi))
        if collected_data:
            process_data(collected_data)
        else:
            messagebox.showwarning("Uyarı", "Hiçbir veri girmediniz!")

    def process_data(data):
        # İşleme fonksiyonlarını burada tanımlayın
        for sayi, ogrenme_ciktisi in data:
            print(f"Sayı: {sayi}, Öğrenme Çıktısı: {ogrenme_ciktisi}")
        messagebox.showinfo("Başarılı", "Veriler başarıyla işlendi!")

    # Dinamik alanlar için çerçeve
    dynamic_fields_frame = ttk.LabelFrame(program_ders_sekmesi, text="Ders Çıktıları Giriş")
    dynamic_fields_frame.pack(fill="x", padx=20, pady=10)

    ttk.Label(dynamic_fields_frame, text="Sayı", width=10).grid(row=0, column=0, padx=10, pady=5)
    ttk.Label(dynamic_fields_frame, text="Öğrenme Çıktısı", width=60).grid(row=0, column=1, padx=10, pady=5)

    dynamic_entry_fields = []
    add_new_row_button = ttk.Button(dynamic_fields_frame, text="Yeni Satır Ekle", command=add_new_row)
    add_new_row_button.grid(row=0, column=2, padx=10, pady=5)
                            
    process_data_button = ttk.Button(dynamic_fields_frame, text="Verileri İşle", command=collect_and_process_data)
    process_data_button.grid(row=0, column=3, padx=10, pady=5)

    # İlk satırı otomatik ekle
    add_new_row()

# Dinamik girdi alanları oluştur
create_dynamic_entry_fields()



def create_dynamic_entry_fields():
    def add_new_row():
        row_idx = len(dynamic_entry_fields)
        sayi_entry = ttk.Entry(dynamic_fields_frame, width=10)
        sayi_entry.grid(row=row_idx + 1, column=0, padx=10, pady=5)
        ogrenme_ciktisi_entry = ttk.Entry(dynamic_fields_frame, width=80)
        ogrenme_ciktisi_entry.grid(row=row_idx + 1, column=1, padx=10, pady=5)
        delete_button = ttk.Button(dynamic_fields_frame, text="Sil", command=lambda: delete_row(row_idx))
        delete_button.grid(row=row_idx + 1, column=2, padx=10, pady=5)
        dynamic_entry_fields.append((sayi_entry, ogrenme_ciktisi_entry, delete_button))

    def delete_row(row_idx):
        dynamic_entry_fields[row_idx][0].destroy()
        dynamic_entry_fields[row_idx][1].destroy()
        dynamic_entry_fields[row_idx][2].destroy()
        dynamic_entry_fields.pop(row_idx)
        # Yeniden düzenle
        for i, (sayi_entry, ogrenme_ciktisi_entry, delete_button) in enumerate(dynamic_entry_fields):
            sayi_entry.grid(row=i + 1, column=0, padx=10, pady=5)
            ogrenme_ciktisi_entry.grid(row=i + 1, column=1, padx=10, pady=5)
            delete_button.grid(row=i + 1, column=2, padx=10, pady=5)

    def collect_and_process_data():
        collected_data = []
        for sayi_entry, ogrenme_ciktisi_entry, _ in dynamic_entry_fields:
            sayi = sayi_entry.get().strip()
            ogrenme_ciktisi = ogrenme_ciktisi_entry.get().strip()
            if sayi and ogrenme_ciktisi:
                collected_data.append((sayi, ogrenme_ciktisi))
        if collected_data:
            process_data(collected_data)
        else:
            messagebox.showwarning("Uyarı", "Hiçbir veri girmediniz!")

    def process_data(data):
        # İşleme fonksiyonlarını burada tanımlayın
        for sayi, ogrenme_ciktisi in data:
            print(f"Sayı: {sayi}, Öğrenme Çıktısı: {ogrenme_ciktisi}")
        messagebox.showinfo("Başarılı", "Veriler başarıyla işlendi!")

    # Dinamik alanlar için çerçeve
    dynamic_fields_frame = ttk.LabelFrame(program_ders_sekmesi, text="Öğrenme Çıktıları Giriş")
    dynamic_fields_frame.pack(fill="x", padx=20, pady=10)

    ttk.Label(dynamic_fields_frame, text="Sayı", width=10).grid(row=0, column=0, padx=10, pady=5)
    ttk.Label(dynamic_fields_frame, text="Öğrenme Çıktısı", width=60).grid(row=0, column=1, padx=10, pady=5)

    dynamic_entry_fields = []
    add_new_row_button = ttk.Button(dynamic_fields_frame, text="Yeni Satır Ekle", command=add_new_row)
    add_new_row_button.grid(row=1, column=0, columnspan=2, pady=10)

    process_data_button = ttk.Button(dynamic_fields_frame, text="Verileri İşle", command=collect_and_process_data)
    process_data_button.grid(row=1, column=2, columnspan=2, pady=10)

    # İlk satırı otomatik ekle
    add_new_row()

def create_program_output_fields():
    def add_new_row():
        row_idx = len(program_output_fields)
        sira_entry = ttk.Entry(program_fields_frame, width=10)
        sira_entry.grid(row=row_idx + 1, column=0, padx=10, pady=5)
        program_ciktisi_entry = ttk.Entry(program_fields_frame, width=80)
        program_ciktisi_entry.grid(row=row_idx + 1, column=1, padx=10, pady=5)
        delete_button = ttk.Button(program_fields_frame, text="Sil", command=lambda: delete_row(row_idx))
        delete_button.grid(row=row_idx + 1, column=2, padx=10, pady=5)
        program_output_fields.append((sira_entry, program_ciktisi_entry, delete_button))

    def delete_row(row_idx):
        program_output_fields[row_idx][0].destroy()
        program_output_fields[row_idx][1].destroy()
        program_output_fields[row_idx][2].destroy()
        program_output_fields.pop(row_idx)
        # Yeniden düzenle
        for i, (sira_entry, program_ciktisi_entry, delete_button) in enumerate(program_output_fields):
            sira_entry.grid(row=i + 1, column=0, padx=10, pady=5)
            program_ciktisi_entry.grid(row=i + 1, column=1, padx=10, pady=5)
            delete_button.grid(row=i + 1, column=2, padx=10, pady=5)

    def collect_and_process_program_data():
        collected_data = []
        for sira_entry, program_ciktisi_entry, _ in program_output_fields:
            sira = sira_entry.get().strip()
            program_ciktisi = program_ciktisi_entry.get().strip()
            if sira and program_ciktisi:
                collected_data.append((sira, program_ciktisi))
        if collected_data:
            process_program_data(collected_data)
        else:
            messagebox.showwarning("Uyarı", "Hiçbir veri girmediniz!")

    def process_program_data(data):
        # İşleme fonksiyonlarını burada tanımlayın
        for sira, program_ciktisi in data:
            print(f"Sıra: {sira}, Program Çıktısı: {program_ciktisi}")
        messagebox.showinfo("Başarılı", "Program çıktıları başarıyla işlendi!")

    # Dinamik alanlar için çerçeve
    program_fields_frame = ttk.LabelFrame(program_ders_sekmesi, text="Program Çıktıları Giriş")
    program_fields_frame.pack(fill="x", padx=20, pady=10)

    ttk.Label(program_fields_frame, text="Sıra", width=10).grid(row=0, column=0, padx=10, pady=5)
    ttk.Label(program_fields_frame, text="Program Çıktısı", width=80).grid(row=0, column=1, padx=10, pady=5)

    program_output_fields = []
    add_new_row_button = ttk.Button(program_fields_frame, text="Yeni Satır Ekle", command=add_new_row)
    add_new_row_button.grid(row=0, column=2, padx=5, pady=10, sticky="w")  # Sol hizalı

    process_data_button = ttk.Button(program_fields_frame, text="Verileri İşle", command=collect_and_process_program_data)
    process_data_button.grid(row=0, column=3, padx=5, pady=10, sticky="w")  # Aynı satırda, sağda


    # İlk satırı otomatik ekle
    add_new_row()

# Program çıktıları için dinamik girdi alanları oluştur
create_program_output_fields()

In [196]:
def tablo3_olustur_butonu_action():
    global tablo2_agirliklar,tablo2_list
    print(tablo2_agirliklar)

    for i in range(len(tablo2_agirliklar)):
        tablo2_agirliklar[i]=int(tablo2_agirliklar[i])/100
    print(tablo2_agirliklar)

    print(tablo2_list)
    global tablo3_list
    for row in tablo2_list:
        multiplication = [tablo2_agirliklar[i] * row[i] for i in range(len(tablo2_agirliklar))]
        tablo3_list.append(multiplication)
    for row in tablo3_list:
        row.append(sum(row))
    print (tablo3_list)

    for i in range(len(tablo3_list)):
        tablo3_list[i].insert(0, tablo2_sırano[i])
    #tablo3-4-5 oluşturulacak
    df = pd.DataFrame(tablo3_list, columns=tablo2_basliklar)

   

    # Excel dosyasına yazdırıyoruz
    df.to_excel('tablo3.xlsx', sheet_name='Tablo3', index=False)

      



In [197]:
def tablo4_olustur_butonu_action():
    global tablo3_list, ogrenci_notlari_tablosu, tablo2_basliklar, ogrenci_basarilari

    # DataFrame'lere dönüştürme
    tablo3_df = pd.DataFrame(tablo3_list, columns=['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM'])
    ogrenci_df = pd.DataFrame(ogrenci_notlari_tablosu, columns=['Ogrenci_No'] + tablo2_basliklar[1:len(tablo2_basliklar)-1])

    # Öğrenci başarılarını saklamak için listeyi temizle
    ogrenci_basarilari.clear()

    def hesapla_ders_notlari(ogrenci_notlari, ogrenci_no):
        sonuc_data = []
        ogrenci_basarisi = [ogrenci_no]

        # TABLO 4 ve öğrenci numarası başlıkları
        sonuc_data.append(['TABLO 4', ogrenci_no] + [''] * (len(tablo2_basliklar) + 2))

        # Sütun başlıkları ekle
        sonuc_data.append(['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM', 'MAX', '% Başarı'])

        for i, row in tablo3_df.iterrows():
            ders_no = i + 1
            toplam_puan = 0
            max_puan = 0

            # Dinamik hesaplamalar
            ders_sonucu = [ders_no]
            for baslik in tablo2_basliklar[1:len(tablo2_basliklar)-1]:
                if baslik in ogrenci_notlari:
                    baslik_puan = ogrenci_notlari[baslik] * row[baslik]
                    toplam_puan += baslik_puan
                    max_puan += row[baslik] * 100
                    ders_sonucu.append(baslik_puan)
                else:
                    ders_sonucu.append(0)

            basari_orani = round((toplam_puan / max_puan) * 100, 2) if max_puan > 0 else 0

            # Başarı oranını listeye ekle
            ogrenci_basarisi.append(basari_orani)

            # Ders satırını tamamla
            ders_sonucu += [round(toplam_puan, 2), max_puan, basari_orani]
            sonuc_data.append(ders_sonucu)

        # Öğrenci başarısını genel listeye ekle
        ogrenci_basarilari.append(ogrenci_basarisi)
        return sonuc_data

    sonuc_data = []
    for i, ogrenci in ogrenci_df.iterrows():
        ogrenci_no = ogrenci['Ogrenci_No']
        ogrenci_sonuc = hesapla_ders_notlari(ogrenci, ogrenci_no)
        sonuc_data.extend(ogrenci_sonuc)

    # Sütun başlıkları
    sonuc_columns = ['Ders Çıktı'] + tablo2_basliklar[1:len(tablo2_basliklar)-1] + ['TOPLAM', 'MAX', '% Başarı']
    sonuc_columns = [''] + sonuc_columns

    try:
        sonuc_df = pd.DataFrame(sonuc_data)
        sonuc_df.to_excel('Tablo4.xlsx', sheet_name='Tüm Öğrenciler', index=False, header=False)
        print("Sonuçlar 'Tablo4.xlsx' dosyasına yazıldı.")

        # Başarıları ayrı bir dosyaya yaz
        basari_df = pd.DataFrame(ogrenci_basarilari)
        basari_df.to_excel('OgrenciBasarilari.xlsx', sheet_name='Başarılar', index=False, header=False)
        print("Başarılar 'OgrenciBasarilari.xlsx' dosyasına yazıldı.")

    except ValueError as e:
        print("Hata:", e)

In [198]:
ders_matrisi={}
def tablo1_yukle_buton_action():
    global ders_matrisi  # Sözlüğe erişmek için global tanımlandı

    # Combobox'tan seçilen dersin adını al
    secilen_ders = ders_sec_combobox7.get()

    if not secilen_ders:  # Ders seçimi kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Uyarı mesajı
        return

    # Dosya yolunu dosya_sec_entry6 widget'ından al
    dosya_adi = dosya_sec_entry6.get()

    if not dosya_adi:  # Dosya yolu kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir dosya seçin.")  # Uyarı mesajı
        return

    try:
        # Excel dosyasını oku
        df = pd.read_excel(dosya_adi, sheet_name=0, engine="openpyxl")

        # Gerekli sütunları seç (ilk 10 satır ve ders çıktıları ile ilişki değerlerini dahil et)
        tablo_verisi = df.iloc[1:, 1:]

        # Matrise dönüştür (liste içinde liste)
        matris = tablo_verisi.values.tolist()
        global tablo1_list
        tablo1_list=matris
        # Matrisi sözlüğe ekle
        ders_matrisi[secilen_ders] = matris

        messagebox.showinfo("Başarılı", f"{secilen_ders} dersi için tablo başarıyla okundu!")  # Başarı mesajı
        print(f"{secilen_ders} dersi için tablo başarıyla okundu")

        # Matris çıktısını yazdır (isteğe bağlı)
        print(f"{secilen_ders} İçeriği (Matris):")
        for satir in matris:
            print(satir)

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")  # Hata mesajı
    except ValueError as e:
        messagebox.showerror("Hata", f"Dosya okuma hatası: {e}")  # Hata mesajı
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")  # Genel hata mesajı


def tablo1_matrisi_goruntule():
    # Eğer ders_matrisi sözlüğü boşsa, uyarı mesajı göster
    if not ders_matrisi:
        messagebox.showwarning("Uyarı", " Lütfen bir des seçin!")  # Ders matrisi yüklenmediyse uyarı
        return

    sozluk_icerigi = ""
    
    # Her ders ve onun matrisi üzerinde döngü
    for ders, matris in ders_matrisi.items():
        # Eğer dersin matrisi boşsa, uyarı mesajı göster
        if not matris:
            messagebox.showwarning("Uyarı", f"{ders} dersinin matrisi boş.")  # Dersin matrisi boşsa uyarı
            continue
        
        sozluk_icerigi += f"Ders: {ders}\nMatris:\n"
        
        # Matrisi satırlara ayırarak, her satırı ekleyin
        for satir in matris:
            sozluk_icerigi += f"{satir}\n"
        
        sozluk_icerigi += "-" * 30 + "\n"  # Her dersin arasına ayırıcı çizgi ekle
    
    # Eğer sozluk_icerigi boşsa, yani hiç ders matrisi gösterilmeyecekse, uyarı mesajı göster
    if not sozluk_icerigi:
        messagebox.showwarning("Uyarı", "Gösterilecek ders matrisi bulunamadı.")  # Matrisi görüntülenebilir ders yoksa uyarı
        return

    # Tüm içerik messagebox içinde gösterilecek
    messagebox.showinfo("Ders Matrisi Sözlüğü", sozluk_icerigi)


In [199]:
def tablo5_olustur_butonu_action():
    global ogrenci_basarilari, tablo1_list

    sonuc_data = []

    for ogrenci in ogrenci_basarilari:
        ogrenci_no = ogrenci[0]
        sonuc_data.append(['TABLO5', ogrenci_no])
        sonuc_data.append(['Prg Çıktı'] + ogrenci[1:] + ['Ortalama'])

        for i, satir in enumerate(tablo1_list[1:], start=1):
            program_cikti = satir[0]
            carpanlar = satir[1:-1]  # Son elemanı da al

            # Başarıları ve çarpanları dinamik olarak eşleştir
            basarilar = ogrenci[1:]
            

            sonuc = [round(b * c, 2) for b, c in zip(basarilar, carpanlar)]
            ortalama = round(sum(sonuc) / len([c for c in carpanlar if c > 0]), 2) if carpanlar else 0

            sonuc_data.append([program_cikti] + sonuc + [ortalama])

        sonuc_data.append([])

    try:
        sonuc_df = pd.DataFrame(sonuc_data)
        sonuc_df.to_excel('Tablo5.xlsx', sheet_name='Tüm Öğrenciler', index=False, header=False)
        print("Sonuçlar 'Tablo5.xlsx' dosyasına yazıldı.")
    except ValueError as e:
        print("Hata:", e)

In [200]:
def tablo2_yukle_buton_action():
    # Dosya yolunu dosya_sec_entry5 widget'ından al
    dosya_adi = dosya_sec_entry5.get()
    secilen_ders = ders_sec_combobox7.get()  # Seçilen ders

    if not dosya_adi:  # Dosya yolu kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir dosya seçin.")  # Uyarı mesajı
        return

    if not secilen_ders:  # Ders seçimi kontrolü
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Uyarı mesajı
        return

    try:
        # Excel dosyasını oku
        df = pd.read_excel(dosya_adi, engine="openpyxl")

        # Tüm veriyi oku (ilk satır başlıkları da dahil)
        tablo_verisi = df  # Verinin tamamını oku

        # Matrise dönüştür (liste içinde liste)
        matris = tablo_verisi.values.tolist()



        global tablo2_basliklar
        tablo2_basliklar=matris[2][1:]
             

        global tablo2_sırano
        tablo2_sırano=[satir[1] for satir in matris[3:]]
        

        global tablo2_agirliklar
        tablo2_agirliklar=matris[1][2:len(matris[0])-1]
        

        global tablo2_list
        tablo2_list=[satir[2:] for satir in matris[3:]]
        print("KONTROL")
        print(tablo2_list)
        print("KONTROL")


        # Seçilen dersle ilişkilendir
        ders_matrisi[secilen_ders] = matris

        # Başarı mesajı
        messagebox.showinfo("Başarılı", f"{secilen_ders} için Tablo2 başarıyla okundu!")
        print(f"{secilen_ders} için Tablo2 başarıyla okundu")

        # Matris çıktısını yazdır (isteğe bağlı)
        print(f"{secilen_ders} Tablo2 İçeriği (Matris):")
        for satir in matris:
            print(satir)

    except FileNotFoundError:
        messagebox.showerror("Hata", f"Dosya bulunamadı: {dosya_adi}")  # Hata mesajı
    except ValueError as e:
        messagebox.showerror("Hata", f"Dosya okuma hatası: {e}")  # Hata mesajı
    except Exception as e:
        messagebox.showerror("Hata", f"Bilinmeyen bir hata oluştu: {e}")  # Genel hata mesajı

def tablo2_matrisi_goruntule():
    secilen_ders = ders_sec_combobox7.get()  # Seçilen dersi al

    # Ders seçimi kontrolü
    if not secilen_ders:
        messagebox.showwarning("Uyarı", "Lütfen bir ders seçin.")  # Ders seçimi yapılmadıysa uyarı
        return

    # Seçilen dersin matrisi kontrolü
    if secilen_ders not in ders_matrisi:  
        messagebox.showwarning("Uyarı", f"{secilen_ders} için bir tablo yüklenmedi!")  # Tablo yüklenmemişse uyarı
        return

    # Seçilen dersin matrisi alınır
    matris = ders_matrisi[secilen_ders]

    # Dersin matrisi boşsa uyarı göster
    if not matris:
        messagebox.showwarning("Uyarı", f"{secilen_ders} dersinin matrisi boş.")  # Dersin matrisi boşsa uyarı
        return

    # Matrisi içeriklerini göster
    matris_metni = "\n".join([", ".join(map(str, satir)) for satir in matris])  # Matrisi stringe çevir
    messagebox.showinfo(f"{secilen_ders} Matrisi", f"{secilen_ders} için matris:\n\n{matris_metni}")




In [201]:
def veritabani_dersleri_getir():
    # Veritabanından derslerin kodunu ve adını çek
    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()
    cursor.execute("SELECT Kodu, Adi FROM Dersler")  # Dersler tablosundan Kodu ve Adi sütunlarını al
    dersler = [(row[0], row[1]) for row in cursor.fetchall()]  # Her bir satırı tuple olarak ekle
    conn.close()
    return dersler

def veritabani_ogrencileri_getir():
    # Veritabanından öğrencileri çek
    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()
    cursor.execute("SELECT No FROM Ogrenciler")
    ogrenciler = [row[0] for row in cursor.fetchall()]
    conn.close()
    return ogrenciler


In [202]:
sekme_defteri.add(tablo_ekleme_sekmesi, text="Tabloları Ekleme Ekranı")



ders_secimi_frame =ttk.LabelFrame(tablo_ekleme_sekmesi,text="Ders Seçimi")
ders_secimi_frame.pack(fill="x",padx=20,pady=10)

tk.Label(ders_secimi_frame, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox7 = ttk.Combobox(ders_secimi_frame, values=veritabani_dersleri_getir())
ders_sec_combobox7.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

# === Tablo1 Yükleme ===
tablo1_frame = ttk.LabelFrame(tablo_ekleme_sekmesi, text="Tablo1 Yükleme")
tablo1_frame.pack(fill="x", padx=20, pady=10)
                        
tk.Label(tablo1_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry6 = ttk.Entry(tablo1_frame)
dosya_sec_entry6.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry6.delete(0, tk.END)
    dosya_sec_entry6.insert(0, dosya)

dosya_sec_butonu6 = ttk.Button(tablo1_frame, text="...", command=dosya_sec)
dosya_sec_butonu6.grid(row=0, column=4, padx=10, pady=5)

# === tablo2 Yükleme ===
tablo2_frame = ttk.LabelFrame(tablo_ekleme_sekmesi, text="Tablo2 Yükleme")
tablo2_frame.pack(fill="x", padx=20, pady=10)

# tablolaları oluştur 
Olusturma_frame = ttk.LabelFrame(tablo_ekleme_sekmesi, text="Tabloları Olustur")
Olusturma_frame.pack(fill="x", padx=20, pady=10)



tk.Label(tablo2_frame, text="Dosya Seç:").grid(row=0, column=2, padx=10, pady=5, sticky="w")
dosya_sec_entry5 = ttk.Entry(tablo2_frame)
dosya_sec_entry5.grid(row=0, column=3, padx=10, pady=5, sticky="ew")

def dosya_sec():
    dosya = filedialog.askopenfilename()
    dosya_sec_entry5.delete(0, tk.END)
    dosya_sec_entry5.insert(0, dosya)

dosya_sec_butonu4 = ttk.Button(tablo2_frame, text="...", command=dosya_sec)
dosya_sec_butonu4.grid(row=0, column=4, padx=10, pady=5)



# Tablo 4 ve Tablo 5 Oluşturma ve Kaydetme
frame4 = tk.Frame(tablo_ekleme_sekmesi)
frame4.pack(pady=10)

tablo1_yukle_butonu = ttk.Button(tablo1_frame, text="Tablo1 Yükle", command=tablo1_yukle_buton_action)
tablo1_yukle_butonu.grid(row=0, column=5, padx=10, pady=5)

tablo2_yukle_butonu = ttk.Button(tablo2_frame, text="Tablo2 Yükle", command=tablo2_yukle_buton_action)
tablo2_yukle_butonu.grid(row=0, column=5, padx=10, pady=5)


tablo3_olustur_butonu = ttk.Button(Olusturma_frame, text="Tablo3 Oluştur", command=tablo3_olustur_butonu_action)
tablo3_olustur_butonu.grid(row=0, column=1, padx=10, pady=5)

tablo4_olustur_butonu = ttk.Button(Olusturma_frame, text="Tablo4 Oluştur", command=tablo4_olustur_butonu_action)
tablo4_olustur_butonu.grid(row=0, column=2, padx=10, pady=5)

tablo5_olustur_butonu = ttk.Button(Olusturma_frame, text="Tablo5 Oluştur", command=None)
tablo5_olustur_butonu.grid(row=0, column=3, padx=10, pady=5)

Manuel_notlar_frame = ttk.LabelFrame(tablo_ekleme_sekmesi, text="Notlar Yükleme")
Manuel_notlar_frame.pack(fill="x", padx=20, pady=10)
                        


In [203]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

sekme_defteri.add(degerlendirme_sekmesi, text="Değerlendirme Kriterleri Ekleme Ekranı")


# Veritabanı bağlantısı
conn = sqlite3.connect("Proje3Database.db")
cursor = conn.cursor()

# Global değişkenler
mevcut_kriterler = []
yuzdeler = []
maksimum_kriter = 5
secilen_ders_id = None



# Ders seçimi için combobox
def ders_secimi_yap():
    global secilen_ders_id
    secilen_ders = ders_combobox.get()
    if secilen_ders:
        cursor.execute("SELECT id FROM Dersler WHERE Adi = ?", (secilen_ders,))
        ders_id = cursor.fetchone()
        if ders_id:
            secilen_ders_id = ders_id[0]
            messagebox.showinfo("Başarılı", f"{secilen_ders} dersi seçildi!")
        else:
            messagebox.showerror("Hata", "Seçilen ders bulunamadı!")

ders_secimi_frame2 =ttk.LabelFrame(degerlendirme_sekmesi,text="Ders Seçimi")
ders_secimi_frame2.pack(fill="x",padx=20,pady=10)

tk.Label(ders_secimi_frame2, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox8 = ttk.Combobox(ders_secimi_frame2, values=veritabani_dersleri_getir())
ders_sec_combobox8.grid(row=0, column=1, padx=10, pady=5, sticky="ew")

# Kriter ve yüzde ekleme
def kriter_ekle():
    kriter = kriter_giris.get()
    try:
        yuzde = int(yuzde_giris.get())
    except ValueError:
        messagebox.showerror("Hata", "Yüzde değeri bir sayı olmalıdır!")
        return

    if kriter in mevcut_kriterler:
        messagebox.showerror("Hata", f"{kriter} zaten eklenmiş!")
        return
    if len(mevcut_kriterler) >= maksimum_kriter:
        messagebox.showerror("Hata", f"Sadece {maksimum_kriter} kriter eklenebilir!")
        return

    mevcut_kriterler.append(kriter)
    yuzdeler.append(yuzde)
    kriter_listesi.insert(tk.END, f"{kriter} - %{yuzde}")

def kriter_sil():
    if len(mevcut_kriterler) > maksimum_kriter:
        mevcut_kriterler.pop()
        yuzdeler.pop()
        kriter_listesi.delete(tk.END)
    else:
        messagebox.showerror("Hata", "Minimum kriter sayısından fazlasını silemezsiniz!")


ders_secimi_frame3 =ttk.LabelFrame(degerlendirme_sekmesi,text="Kriter İşlemleri")
ders_secimi_frame3.pack(fill="x",padx=20,pady=10)

tk.Label(ders_secimi_frame3, text=":").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox9 = ttk.Combobox(ders_secimi_frame3, values=veritabani_dersleri_getir())
ders_sec_combobox9.grid(row=0, column=1, padx=10, pady=5, sticky="ew")


kriter_giris_label = tk.Label(degerlendirme_sekmesi, text="Kriter:")
kriter_giris_label.pack()
kriter_giris = tk.Entry(degerlendirme_sekmesi)
kriter_giris.pack()

yuzde_giris_label = tk.Label(degerlendirme_sekmesi, text="Yüzde:")
yuzde_giris_label.pack()
yuzde_giris = tk.Entry(degerlendirme_sekmesi)
yuzde_giris.pack()

kriter_ekle_button = tk.Button(degerlendirme_sekmesi, text="Kriter Ekle", command=kriter_ekle)
kriter_ekle_button.pack()

kriter_sil_button = tk.Button(degerlendirme_sekmesi, text="Kriter Sil", command=kriter_sil)
kriter_sil_button.pack()

# Kriter listesi
kriter_listesi_label = tk.Label(degerlendirme_sekmesi, text="Kriter Listesi:")
kriter_listesi_label.pack()
kriter_listesi = tk.Listbox(degerlendirme_sekmesi)
kriter_listesi.pack()

# Yüzde kontrol ve kaydetme
def yuzde_kontrol_ve_kaydet():
    toplam = sum(yuzdeler)
    if toplam != 100:
        messagebox.showerror("Hata", "Yüzdelikleri düzenleyin, toplamları 100 olmalı!")
        return

    for kriter, yuzde in zip(mevcut_kriterler, yuzdeler):
        cursor.execute("""
            INSERT INTO PuanTuru (Tur, DersId, Yuzdesi)
            VALUES (?, ?, ?)
        """, (kriter, secilen_ders_id, yuzde))
    
    conn.commit()
    messagebox.showinfo("Başarılı", "Kriterler başarıyla kaydedildi!")
    kriter_listesi.delete(0, tk.END)
    mevcut_kriterler.clear()
    yuzdeler.clear()

kaydet_button = tk.Button(degerlendirme_sekmesi, text="Kaydet", command=yuzde_kontrol_ve_kaydet)
kaydet_button.pack()

# Düzenleme
def kriter_duzenle():
    try:
        secilen_indeks = kriter_listesi.curselection()[0]
        eski_kriter = mevcut_kriterler[secilen_indeks]
        yeni_kriter = kriter_giris.get()
        yeni_yuzde = int(yuzde_giris.get())
        mevcut_kriterler[secilen_indeks] = yeni_kriter
        yuzdeler[secilen_indeks] = yeni_yuzde
        kriter_listesi.delete(secilen_indeks)
        kriter_listesi.insert(secilen_indeks, f"{yeni_kriter} - %{yeni_yuzde}")
    except IndexError:
        messagebox.showerror("Hata", "Düzenlemek için bir kriter seçin!")
    except ValueError:
        messagebox.showerror("Hata", "Yüzde değeri bir sayı olmalıdır!")

duzenle_button = tk.Button(degerlendirme_sekmesi, text="Düzenle", command=kriter_duzenle)
duzenle_button.pack()



In [204]:
sekme_defteri.add(not_yukleme_sekmesi, text="Not Yükleme Ekranı")

def veritabani_ogrencileri_getir():
    """Veritabanından öğrencileri çeker"""
    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()
    cursor.execute("SELECT No FROM Ogrenciler")
    ogrenciler = [row[0] for row in cursor.fetchall()]
    conn.close()
    return ogrenciler


def notlari_kaydet():
    """Girilen notları veritabanına kaydeder"""
    ogrenci_no = ogrenci_combobox.get()
    ders_id = 1  # Örnek ders ID'si. İhtiyacınıza göre dinamik hale getirebilirsiniz.
    notlar = {
        "Odev1": odev1_entry.get(),
        "Odev2": odev2_entry.get(),
        "Quiz1": quiz1_entry.get(),
        "Quiz2": quiz2_entry.get(),
        "Vize": vize_entry.get(),
        "Final": final_entry.get(),
    }

    if not ogrenci_no:
        print("Lütfen bir öğrenci seçin.")
        return

    conn = sqlite3.connect("Proje3Database.db")
    cursor = conn.cursor()

    # Öğrenci ID'sini bul
    cursor.execute("SELECT id FROM Ogrenciler WHERE No = ?", (ogrenci_no,))
    ogrenci_id = cursor.fetchone()
    if not ogrenci_id:
        print("Seçilen öğrenci bulunamadı.")
        conn.close()
        return

    ogrenci_id = ogrenci_id[0]

    # Notları kaydet
    for tur, puan in notlar.items():
        if puan:  # Sadece girilmiş notları kaydet
            cursor.execute(
                """
                INSERT INTO Notlar (OgrenciId, DersId, Puan, Tur)
                VALUES (?, ?, ?, ?)
                """,
                (ogrenci_id, ders_id, puan, tur),
            )

    conn.commit()
    conn.close()
    print("Notlar başarıyla kaydedildi.")

#Ders seçimi frame
ders_secimi_frame4 =ttk.LabelFrame(not_yukleme_sekmesi,text="Ders Seçimi")
ders_secimi_frame4.pack(fill="x",padx=20,pady=10)

tk.Label(ders_secimi_frame4, text="Ders Seç:").grid(row=0, column=0, padx=10, pady=5, sticky="w")
ders_sec_combobox10 = ttk.Combobox(ders_secimi_frame4, values=veritabani_dersleri_getir())
ders_sec_combobox10.grid(row=0, column=1, padx=10, pady=5, sticky="ew")



# Frame
not_girisi_frame = ttk.LabelFrame(not_yukleme_sekmesi, text="Not Girişi")
not_girisi_frame.pack(fill="both", expand=True, padx=20, pady=10)

# Öğrenci Combobox
ogrenci_label = ttk.Label(not_girisi_frame, text="Öğrenci")
ogrenci_label.grid(row=0, column=0, padx=10, pady=5)

ogrenci_combobox = ttk.Combobox(
    not_girisi_frame, values=veritabani_ogrencileri_getir(), state="readonly"
)
ogrenci_combobox.grid(row=0, column=1, padx=10, pady=5)

# Not Girişi Alanları
odev1_label = ttk.Label(not_girisi_frame, text="Ödev 1")
odev1_label.grid(row=1, column=0, padx=10, pady=5)
odev1_entry = ttk.Entry(not_girisi_frame)
odev1_entry.grid(row=1, column=1, padx=10, pady=5)

odev2_label = ttk.Label(not_girisi_frame, text="Ödev 2")
odev2_label.grid(row=1, column=2, padx=10, pady=5)
odev2_entry = ttk.Entry(not_girisi_frame)
odev2_entry.grid(row=1, column=3, padx=10, pady=5)

quiz1_label = ttk.Label(not_girisi_frame, text="Quiz 1")
quiz1_label.grid(row=2, column=0, padx=10, pady=5)
quiz1_entry = ttk.Entry(not_girisi_frame)
quiz1_entry.grid(row=2, column=1, padx=10, pady=5)

quiz2_label = ttk.Label(not_girisi_frame, text="Quiz 2")
quiz2_label.grid(row=2, column=2, padx=10, pady=5)
quiz2_entry = ttk.Entry(not_girisi_frame)
quiz2_entry.grid(row=2, column=3, padx=10, pady=5)

vize_label = ttk.Label(not_girisi_frame, text="Vize")
vize_label.grid(row=3, column=0, padx=10, pady=5)
vize_entry = ttk.Entry(not_girisi_frame)
vize_entry.grid(row=3, column=1, padx=10, pady=5)

final_label = ttk.Label(not_girisi_frame, text="Final")
final_label.grid(row=3, column=2, padx=10, pady=5)
final_entry = ttk.Entry(not_girisi_frame)
final_entry.grid(row=3, column=3, padx=10, pady=5)

# Kaydet Butonu
kaydet_butonu = ttk.Button(not_girisi_frame, text="Kaydet", command=notlari_kaydet)
kaydet_butonu.grid(row=4, column=1, columnspan=2,pady=10)

In [205]:
pencere.mainloop()